# Train and test the seq2seq model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random
import json
import os
import time


import tensorflow as tf

import tensorflow.contrib.legacy_seq2seq as seq2seq

from sklearn.model_selection import train_test_split

In [2]:
xo = np.load("iseq_n.npy")
yo = np.load("oseq_n.npy")

In [3]:
x = xo[:,:26,:]
y = yo[:,:26]

In [4]:
nxchars = x.shape[2]
print(nxchars)
ltokens = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '+', '=', '#leq', '#neq', '#geq', '#alpha',
                            '#beta', '#lambda', '#lt', '#gt', 'x', 'y', '^', '#frac', '{', '}' ,' ']
print("#ltokens: ", len(ltokens))

30
#ltokens:  28


In [5]:
x_seq_length = len(x[0])
y_seq_length = len(y[0])- 1
print(x_seq_length, y_seq_length)

26 25


In [6]:
def batch_data(x, y, batch_size):
    shuffle = np.random.permutation(len(x))
    start = 0
#     from IPython.core.debugger import Tracer; Tracer()()
    x = x[shuffle]
    y = y[shuffle]
    while start + batch_size <= len(x):
        yield x[start:start+batch_size], y[start:start+batch_size]
        start += batch_size

In [7]:
batch_size = 512
nodes = 256
embed_size = 20

# Tensor where we will feed the data into graph
inputs = tf.placeholder(tf.float32, (None, x_seq_length, nxchars), 'inputs')
outputs = tf.placeholder(tf.int32, (None, None), 'output')
targets = tf.placeholder(tf.int32, (None, None), 'targets')

# Embedding layers
output_embedding = tf.Variable(tf.random_uniform((len(ltokens)+1, embed_size), -1.0, 1.0), name='dec_embedding')
date_output_embed = tf.nn.embedding_lookup(output_embedding, outputs)

with tf.variable_scope("encoding") as encoding_scope:
    lstm_enc = tf.contrib.rnn.BasicLSTMCell(nodes)
    _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=inputs, dtype=tf.float32)

with tf.variable_scope("decoding") as decoding_scope:
    lstm_dec = tf.contrib.rnn.BasicLSTMCell(nodes)
    dec_outputs, _ = tf.nn.dynamic_rnn(lstm_dec, inputs=date_output_embed, initial_state=last_state)
#connect outputs to 
logits = tf.contrib.layers.fully_connected(dec_outputs, num_outputs=len(ltokens)+1, activation_fn=None) 
with tf.name_scope("optimization"):
    # Loss function
    loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, y_seq_length]))
    # Optimizer
    optimizer = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


Train the graph above:

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=36)
print(y_train[0])

[28  6  3 17 23  3 10  6  1  5 27 15 27 10  5  9  9 27 27 27 27 27 27 27
 27 27]


In [9]:
def save(sess):
    saver = tf.train.Saver(None)

    save_path = saver.save(sess, save_path="seq_mod/model", global_step=None)
    print('model saved at %s' % save_path)

In [10]:
def restore(sess):
    saver = tf.train.Saver(None)
    path = "seq_mod/model"
    saver.restore(sess, save_path=path)
    print('model restored from %s' % path)

In [ ]:
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    restore(sess)
    epochs = 3600
    for epoch_i in range(epochs):
        start_time = time.time()
        for batch_i, (source_batch, target_batch) in enumerate(batch_data(X_train, y_train, batch_size)):
            _, batch_loss, batch_logits = sess.run([optimizer, loss, logits],
                feed_dict = {inputs: source_batch,
                 outputs: target_batch[:, :-1],
                 targets: target_batch[:, 1:]})

        accuracy = np.mean(batch_logits.argmax(axis=-1) == target_batch[:,1:])
        print('Epoch {:3} Loss: {:>6.3f} Accuracy: {:>6.4f} Epoch duration: {:>6.3f}s'.format(epoch_i, batch_loss, 
                                                                          accuracy, time.time() - start_time))

        source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

        dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
        for i in range(y_seq_length):
            batch_logits = sess.run(logits,
                        feed_dict = {inputs: source_batch,
                         outputs: dec_input})
            prediction = batch_logits[:,-1].argmax(axis=-1)
            dec_input = np.hstack([dec_input, prediction[:,None]])

        print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))     
        if epoch_i % 5 == 0:
            save(sess)
    
    save(sess)

INFO:tensorflow:Restoring parameters from seq_mod/model
model restored from seq_mod/model
Epoch   0 Loss:  1.473 Accuracy: 0.8372 Epoch duration: 23.782s
Accuracy on test set is:  0.759
model saved at seq_mod/model
Epoch   1 Loss:  1.327 Accuracy: 0.8366 Epoch duration: 25.597s
Accuracy on test set is:  0.755
Epoch   2 Loss:  1.230 Accuracy: 0.8367 Epoch duration: 23.353s
Accuracy on test set is:  0.745
Epoch   3 Loss:  1.078 Accuracy: 0.8430 Epoch duration: 23.349s
Accuracy on test set is:  0.743
Epoch   4 Loss:  1.109 Accuracy: 0.8398 Epoch duration: 23.323s
Accuracy on test set is:  0.741
Epoch   5 Loss:  1.002 Accuracy: 0.8460 Epoch duration: 23.539s
Accuracy on test set is:  0.730
model saved at seq_mod/model
Epoch   6 Loss:  0.950 Accuracy: 0.8452 Epoch duration: 23.502s
Accuracy on test set is:  0.716
Epoch   7 Loss:  0.911 Accuracy: 0.8460 Epoch duration: 23.406s
Accuracy on test set is:  0.722
Epoch   8 Loss:  0.833 Accuracy: 0.8541 Epoch duration: 23.479s
Accuracy on test set

Accuracy on test set is:  0.648
Epoch  80 Loss:  0.262 Accuracy: 0.9173 Epoch duration: 24.390s
Accuracy on test set is:  0.651
model saved at seq_mod/model
Epoch  81 Loss:  0.235 Accuracy: 0.9237 Epoch duration: 23.536s
Accuracy on test set is:  0.649
Epoch  82 Loss:  0.258 Accuracy: 0.9164 Epoch duration: 23.502s
Accuracy on test set is:  0.646
Epoch  83 Loss:  0.243 Accuracy: 0.9198 Epoch duration: 23.650s
Accuracy on test set is:  0.651
Epoch  84 Loss:  0.288 Accuracy: 0.9051 Epoch duration: 23.774s
Accuracy on test set is:  0.638
Epoch  85 Loss:  0.267 Accuracy: 0.9098 Epoch duration: 23.628s
Accuracy on test set is:  0.644
model saved at seq_mod/model
Epoch  86 Loss:  0.228 Accuracy: 0.9254 Epoch duration: 23.532s
Accuracy on test set is:  0.651
Epoch  87 Loss:  0.225 Accuracy: 0.9261 Epoch duration: 25.120s
Accuracy on test set is:  0.647
Epoch  88 Loss:  0.214 Accuracy: 0.9322 Epoch duration: 24.030s
Accuracy on test set is:  0.648
Epoch  89 Loss:  0.208 Accuracy: 0.9347 Epoch 

Translate on test set

In [ ]:
with tf.Session() as sess: 
    restore(sess)
    batch_size = 512
    source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

    dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
    for i in range(y_seq_length):
        batch_logits = sess.run(logits,
                    feed_dict = {inputs: source_batch,
                     outputs: dec_input})
        prediction = batch_logits[:,-1].argmax(axis=-1)
        dec_input = np.hstack([dec_input, prediction[:,None]])

    print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))

Let's randomly take two from this test set and see what it spits out:

In [ ]:
i = 102
print(dec_input[i,:])
print(ltokens)
seq = ""
for c in dec_input[i,1:]:
    c = int(c)
    if c != 28:
        seq += ltokens[c] 
        
print(seq)

seq = ""
for c in target_batch[i,1:]:
    c = int(c)
    
    if c != 28:
        seq += ltokens[c] 

print("Correct: ", seq)

In [ ]:
with tf.Session() as sess: 
    restore(sess)
    batch_size = 512
    source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

    dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
    for i in range(y_seq_length):
        batch_logits = sess.run(logits,
                    feed_dict = {inputs: source_batch,
                     outputs: dec_input})
        prediction = batch_logits[:,-1].argmax(axis=-1)
        dec_input = np.hstack([dec_input, prediction[:,None]])

    print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))
    cc = 0
    fcc = 0
    fc = 0
    fl = 0
    nl = 0
    for i in range(len(dec_input)):
        feq = False
        pseq = ""
        for c in dec_input[i,1:]:
            c = int(c)
            if c != 28:
                pseq += ltokens[c] 

        cseq = ""
        cseql = ""
        for c in target_batch[i,1:]:
            c = int(c)

            if c != 28:
                cseq += ltokens[c] 
                cseql += ltokens[c][0]
                if ltokens[c] == "#frac":
                    fc += 1
                    feq = True

        if pseq == cseq:
            cc += 1
            if feq:
                fcc += 1
        cseql = cseql.rstrip()
        if feq:
            fl += len(cseql)
        else:
            nl += len(cseql)
        
    print("Accuracy %.2f %%" % (cc/len(dec_input)*100))  
    print("Accuracy for fraction equations %.2f %%" % (fcc/fc*100))
    print("Accuracy for simple equations %.2f %%" % ((cc-fcc)/(len(dec_input)-fc)*100))
    print("Average length frac: %.1f" % (fl/fc))
    print("Average length simple: %.1f" % (nl/(len(dec_input)-fc)))